In [40]:
import gpxpy
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from datetime import timedelta
from geopy.distance import geodesic

In [43]:
def create_actiity_gdf(filepath):
    
    times = []
    timedeltas = []
    geometries = []
    distances = []
    paces = []
    
    with open(filepath, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
        name = ''
        type = ''
        dateandTime = ''
        for track in gpx.tracks:
           name = track.name
           type = track.type
           for segment in track.segments:
               points = segment.points
               previous_point = None
               
               for i, point in enumerate(points):
                   
                   if point.time is None:
                    continue
                   
                   # Extract time and create Point geometry (XY only)
                   time = point.time  # datetime object
                   geometry = Point(point.longitude, point.latitude)  # Shapely Point object
    
                   # Calculate timedelta and distance from the previous point
                   if previous_point:
                       timedelta_between = time - previous_point.time  # timedelta object
                       timedeltas.append(timedelta_between)
                       
                       # Calculate the distance in meters between current and previous points
                       distance_between = geodesic(
                           (previous_point.latitude, previous_point.longitude),
                           (point.latitude, point.longitude)
                       ).meters
                       distances.append(distance_between)
                       
                       # Calculate pace (minutes per mile)
                       if distance_between > 0:  # Avoid division by zero
                           pace = (timedelta_between.total_seconds() / 60) / (distance_between / 1609.34)  # minutes per mile
                       else:
                           pace = 0
                       paces.append(pace)
                   else:
                       # For the first point, set timedelta, distance, and pace to 0
                       timedeltas.append(timedelta(0))
                       distances.append(0)
                       paces.append(0)
    
                   # Append extracted data
                   times.append(time)
                   geometries.append(geometry)
                   
                   # Update previous point
                   previous_point = point
    
    gdf = gpd.GeoDataFrame({
        'time': times,
        'timedelta': timedeltas,
        'geometry': geometries,
        'distance': distances,
        'pace': paces
    }, geometry='geometry', crs="EPSG:4326")

    return gdf, name, type
    

IndentationError: expected an indented block after 'if' statement on line 12 (2133143098.py, line 14)

,time,timedelta,geometry,distance,pace
0,2023-08-27 16:55:12+00:00,0 days 00:00:00,POINT (-94.16936 36.07735),0.000000,0.000000
1,2023-08-27 16:55:13+00:00,0 days 00:00:01,POINT (-94.16936 36.07733),1.594683,16.819857
2,2023-08-27 16:55:14+00:00,0 days 00:00:01,POINT (-94.16937 36.07732),1.473271,18.205979
3,2023-08-27 16:55:15+00:00,0 days 00:00:01,POINT (-94.16937 36.0773),2.413971,11.111291
4,2023-08-27 16:55:16+00:00,0 days 00:00:01,POINT (-94.16937 36.07726),4.439331,6.041976
...,...,...,...,...,...
4552,2023-08-27 18:17:00+00:00,0 days 00:00:01,POINT (-94.16875 36.07683),1.892517,14.172838
4553,2023-08-27 18:17:01+00:00,0 days 00:00:01,POINT (-94.16873 36.07681),2.468453,10.866049
4554,2023-08-27 18:17:02+00:00,0 days 00:00:01,POINT (-94.16873 36.0768),0.958019,27.997691
4555,2023-08-27 18:17:03+00:00,0 days 00:00:01,POINT (-94.16873 36.0768),0.519633,51.617796
